In [ ]:
hidden_channels=64
final_layer_dim=hidden_channels/4
class GNN_Lstm(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels,requires_grad=True)
        self.actfn1=nn.ReLU()
        
        self.conv2 = SAGEConv((-1, -1),int( hidden_channels/2),requires_grad=True)
        self.actfn2=nn.ReLU()
        
        self.conv3 = SAGEConv((-1, -1),int( final_layer_dim),requires_grad=True)
        self.actfn3=nn.ReLU()
        
        self.conv4 = SAGEConv((-1, -1),int( hidden_channels/8),requires_grad=True)
        self.actfn4=nn.ReLU()
        
        
        self.sigmoid= nn.Sigmoid()

    def forward(self, x, edge_index):
        
        x = self.actfn1(self.conv1(x, edge_index))
        #x= F.dropout(x,p=.5,training=self.training)
        
        x = self.actfn2(self.conv2(x, edge_index))
        #x= F.dropout(x,p=.7,training=self.training)
        
        x = self.actfn3(self.conv3(x, edge_index))
        
        #out = self.actfn4(self.conv4(x, edge_index))
        
        #out= F.dropout(x,p=.7,training=self.training)
        #pool={key: gmp(out[key].x, out[key].batch) for key in out.node_types}
        #out=self.sigmoid( self.out(x))
        #ic(pool)
        
        #pool=gmp(out)
        return x

In [ ]:
lookback=3
input_size=33
out_channels=1
num_layers=1
hidden_size=20
class Classifer_Lstm(nn.Module):
    
    def __init__(self ):
        super().__init__()
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        self.sigmoid=nn.Sigmoid()
        
        self.lstm = nn.LSTM(
            input_size=self.input_size ,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self,x):
        #print(x.shape)
        h0 = torch.zeros(self.num_layers, 1, self.hidden_size).requires_grad_()
        # Initialize long-term memory
        c0 = torch.zeros(self.num_layers, 1, self.hidden_size).requires_grad_()
        # Pass all inputs to lstm layer
        out, _ = self.lstm(x, (h0, c0))
        #print(out.shape)
        out = self.sigmoid(self.fc(out[:, -1, :]))
        
        return out

In [ ]:
class GNN_LSTM_Classifier(nn.Module):
    def __init__(self):
        super(GNN_LSTM_Classifier, self).__init__()
        
        self.model_embedding=GNN_Lstm()
        self.model_embedding = to_hetero(self.model_embedding, data.metadata(),aggr='max')
        self.classifer = Classifer_Lstm()
        #lo,atom,prev_answer,prev_lo,prev_atom,prev_answer1,prev_lo1,prev_atom1
    def forward(self, x,edges,batch_num,lookback,question_sequence):
        node_embeddings = self.model_embedding(x,edges)
        rnn_input_list=[]
        #features_concat_list=[]
        
        for i in range(lookback+1):
            
            #print([i,batch_num-i])
            lo=question_sequence[0][batch_num-i]
            atom=question_sequence[1][batch_num-i]
            prev_answer=torch.tensor([question_sequence[2][batch_num-1-i]]).float()

            features_concat=torch.cat([node_embeddings['atom'][atom],node_embeddings['lo'][lo],prev_answer])
            rnn_input_list.append(features_concat)
        
                                         
            
        #user_embeddings=torch.cat([node_embeddings['atom'][atom] , node_embeddings['lo'][lo],prev_answer,node_embeddings['atom'][prev_atom] , node_embeddings['lo'][prev_lo],prev_answer1,node_embeddings['atom'][prev_atom1] , node_embeddings['lo'][prev_lo1]])  
       
        rnn_input_stacked=torch.stack(rnn_input_list).view(1,lookback+1,features_concat.shape[0])

        rnn_outputs = self.classifer(rnn_input_stacked)
    
        return rnn_outputs,node_embeddings

In [ ]:
training_size=sum([users_question_sequence_train_lists[i].shape[1] for i in range(len(users_question_sequence_train_lists))])

model=GNN_LSTM_Classifier()
learning_rate=0.001
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=5e-12)
criterion=torch.nn.BCELoss()
#criterion=F.binary_cross_entropy
train_acc_mean_list=[]
test_acc_mean_list=[]
loss_list=[]
accuracy_list=[]
best_score=0

torch.manual_seed(42)
loss_moving_avg=[]
user_embedings=[]
for epoch_num in range(2):
    epoch_accuracy=[]
    out_list=[]
    total_loss=0
    
    prob=[]
    target=[]
    for user_count,question_sequence in enumerate(users_question_sequence_train_lists):
        question_sequence_len=question_sequence.shape[1]
        batch_loss=0
        for  batch_num in range(lookback,question_sequence_len):

            model.train()

            optimizer.zero_grad()   
            #x_dict = {key: gmp(batch[key].x, batch[key].batch) for key in batch.node_types}


            answer=torch.tensor([question_sequence[2][batch_num]]).float()


            out,node_embeddings=model(data.x_dict, data.edge_index_dict,batch_num,lookback,question_sequence) 

            loss=F.binary_cross_entropy(out.view(1),answer)
            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())
            total_loss=total_loss+loss.item()
            train_correct=out.round()==answer

            train_acc=train_correct*1
            accuracy=(train_correct*1).item()
            epoch_accuracy.append(accuracy)
            #accuracy_list.append(accuracy)
            prob.append(out.item())
            target.append(answer.item())
            batch_loss=batch_loss+loss.item()
        print('epoch {} | batch {} | batch loss {:.5f}'.format(epoch_num,user_count+1,batch_loss/question_sequence_len))
    
    performance='epoch {} | loss {:.5f} | accuracy {:.5f} | auc {:.5f} \n'.format(epoch_num,total_loss/(training_size),np.mean(epoch_accuracy),roc_auc_score(np.array(target),prob))
    print('-'*len(performance))
    print(performance)
    
    loss_moving_avg.append(total_loss/(batch_num+1))
        

In [ ]:

with torch.no_grad():
   
    for user_count,question_sequence in enumerate(users_question_sequence_test_lists):
            question_sequence_len=question_sequence.shape[1]
            batch_accuracy=[]
            batch_target=[]
            batch_prob=[]
            for  batch_num in range(lookback,question_sequence_len):

                model.eval()
                answer=torch.tensor([question_sequence[2][batch_num]]).float()

                out,node_embeddings=model(data.x_dict, data.edge_index_dict,batch_num,lookback,question_sequence) 

                train_correct=out.round()==answer

                train_acc=train_correct*1
                accuracy=(train_correct*1).item()
                epoch_accuracy.append(accuracy)
                prob.append(out.item())
                target.append(answer.item())
                batch_target.append(answer.item())
                batch_prob.append(out.item() )
                batch_accuracy.append(accuracy)
            batch_performance='accuracy {:.5f} | auc {:.5f} \n'.format(np.mean(batch_accuracy),roc_auc_score(np.array(batch_target),batch_prob))
                
            print(batch_performance)
      
    performance='accuracy {:.5f} | auc {:.5f} \n'.format(np.mean(epoch_accuracy),roc_auc_score(np.array(target),prob))
    print('-'*len(performance))
    print(performance)
